In [ ]:
import os
import shutil
 
# To delete all files in the /kaggle/working/ directory
folder_path = '/kaggle/working'
 
# Check if the folder exists
if os.path.exists(folder_path):
    # Delete all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f'All files in {folder_path} have been deleted.')
else:
    print(f'{folder_path} does not exist.')

In [2]:
import shutil
import os

# Define the source and destination directories
source_dir = '/kaggle/input/chatbot-new/'
destination_dir = '/kaggle/working/'

# Copy all files from source to destination, maintaining the directory structure
for dirname, _, filenames in os.walk(source_dir):
    for filename in filenames:
        # Construct the full file paths
        source_file = os.path.join(dirname, filename)
        # Create the corresponding destination directory
        relative_path = os.path.relpath(dirname, source_dir)
        destination_subdir = os.path.join(destination_dir, relative_path)
        os.makedirs(destination_subdir, exist_ok=True)
        destination_file = os.path.join(destination_subdir, filename)
        # Copy the file
        shutil.copy(source_file, destination_file)

print("All files have been copied to /kaggle/working/")


All files have been copied to /kaggle/working/


In [3]:
!pip install datasets transformers scikit-learn torch accelerate rouge-score nltk evaluate tensorboard tensorboardX

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e4db559f2825fb207971cb65df982b2b6885d0f03cb067715097acdb4b886e7d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [4]:
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk
import evaluate
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, default_data_collator, EarlyStoppingCallback, TrainerCallback, AutoModelForCausalLM
from torch.utils.data import DataLoader
import numpy as np
import torch
import re

In [5]:
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base").to(device)

In [ ]:
# Define preprocessing functions for dataset1
def extract_text(data):
    parts = data.split("Text:")
    extracted_texts = []
    for part in parts[1:]:
        text = part.split("Document:")[0].strip()
        extracted_texts.append(text)
    updated_data = "\n".join(extracted_texts)
    return updated_data
 
def process_text(data):
    if 'Cited Documents: None' in data:
        return None
    answer_start = data.find('Answer:') + len('Answer:')
    answer_text = data[answer_start:].strip()
    cleaned_text = re.sub(r'<[^>]*>', '', answer_text)
    return cleaned_text
 
def preprocess_function_dataset1(examples):
    questions = examples["question"]
    documents = examples["documents"]
    targets = examples["answer"]
 
    # Apply text extraction and processing
    texts = [extract_text(d) for d in documents]
    processed_targets = [process_text(t) for t in targets]
 
    # Filter out examples with invalid targets
    valid_indices = [i for i, target in enumerate(processed_targets) if target is not None]
    questions = [questions[i] for i in valid_indices]
    texts = [texts[i] for i in valid_indices]
    targets = [processed_targets[i] for i in valid_indices]
 
    # Filter out documents with more than 1000 words
    valid_indices = [i for i, text in enumerate(texts) if len(text.split()) <= 1000]
    questions = [questions[i] for i in valid_indices]
    texts = [texts[i] for i in valid_indices]
    targets = [targets[i] for i in valid_indices]
 
    # Tokenize questions and texts separately (without padding yet)
    # question_encodings = tokenizer(questions, padding=False, truncation=False)
    # text_encodings = tokenizer(texts, padding=False, truncation=False)
 
    # Filter out examples exceeding max length *before* padding or truncation
    inputs = []
    final_targets = []

    for q, t, target in zip(questions, texts, targets):
        input_text = f"Answer the following question based on the provided text:\n\nQuestion: {q} \n\nText: {t}"
        encodings = tokenizer(input_text, padding=False, truncation=False)

        if len(encodings.input_ids) <= 1024:
            inputs.append(encodings)
            final_targets.append(target)
 
    # Now pad the filtered examples
    model_inputs = tokenizer.pad(inputs, padding="max_length", max_length=1024, return_tensors="pt")
 
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(final_targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Define preprocessing function for dataset2
def preprocess_function_dataset2(examples):
    questions = examples["question"]
    documents = examples["documents"]
    targets = examples["answer"]
 
    # Ensure documents, questions, and targets are strings
    documents = [str(doc) for doc in documents]
    questions = [str(q) for q in questions]
    targets = [str(target) for target in targets]
 
    # Filter out documents with more than 1000 words
    valid_indices = [i for i, doc in enumerate(documents) if len(doc.split()) <= 1000]
    questions = [questions[i] for i in valid_indices]
    documents = [documents[i] for i in valid_indices]
    targets = [targets[i] for i in valid_indices]
 
    # Tokenize questions and documents separately (without padding yet)
    # question_encodings = tokenizer(questions, padding=False, truncation=False)
    # document_encodings = tokenizer(documents, padding=False, truncation=False)
 
    # Filter out examples exceeding max length *before* padding or truncation
    inputs = []
    final_targets = []

    for q, d, target in zip(questions, documents, targets):
        input_text = f"Answer the following question based on the provided text:\n\nQuestion: {q} \n\nText: {d}"
        encodings = tokenizer(input_text, padding=False, truncation=False)

        if len(encodings.input_ids) <= 1024:
            inputs.append(encodings)
            final_targets.append(target)
 
    # Now pad the filtered examples
    model_inputs = tokenizer.pad(inputs, padding="max_length", max_length=1024, return_tensors="pt")
 
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(final_targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Load and Prepare Dataset 1
dataset1 = load_dataset("glaiveai/RAG-v1")
 
# Filter out examples with answer_mode = "Mixed" in dataset1
dataset1 = dataset1.filter(lambda example: example['answer_mode'] == 'Grounded')

In [ ]:
# Load and Prepare Dataset 1
dataset2 = load_dataset("neural-bridge/rag-dataset-12000")

# Rename columns in dataset2 to match dataset1
dataset2 = dataset2.rename_column("context", "documents")

In [ ]:
# Preprocess dataset1
dataset1 = dataset1.map(preprocess_function_dataset1, batched=True, remove_columns=['documents', 'answer', 'question', 'system_prompt', 'answer_mode'])


In [ ]:
# Preprocess dataset2
dataset2 = dataset2.map(preprocess_function_dataset2, batched=True, remove_columns=['documents', 'question', 'answer'])

In [ ]:
# Combine the datasets
combined_dataset = concatenate_datasets([dataset1["train"], dataset2["train"], dataset2["test"]])
 
# Shuffle the combined dataset
combined_dataset = combined_dataset.shuffle(seed=42)

In [ ]:
# Split the combined dataset
train_dataset, temp_dataset = combined_dataset.train_test_split(test_size=0.2, seed=42).values()
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.5, seed=42).values()

In [ ]:
# Count the number of examples in each split
print(f"Number of examples in train dataset: {len(train_dataset)}")
print(f"Number of examples in validation dataset: {len(val_dataset)}")
print(f"Number of examples in test dataset: {len(test_dataset)}")

In [ ]:
# Save the splits to your directory
train_dataset.save_to_disk("train_dataset")
val_dataset.save_to_disk("val_dataset")
test_dataset.save_to_disk("test_dataset")

In [9]:
# Load the splits from your directory
train_dataset = load_from_disk("train_dataset")
val_dataset = load_from_disk("val_dataset")
test_dataset = load_from_disk("test_dataset")

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.train()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
#     eval_steps=1000,
#     save_steps=1000,
    save_total_limit=4,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="all",
    fp16=True,
    max_grad_norm=1.0
)

In [ ]:
import torch
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    optimizers=(optimizer, None)
)

checkpoint_path = "/kaggle/working/results/checkpoint-11193"
trainer.train(resume_from_checkpoint=checkpoint_path)
# trainer.train()

In [ ]:
trainer.save_model("./best_model")

In [ ]:
first_test_example = test_dataset[0]

# Decode and print the question and text
input_ids = first_test_example["input_ids"]
decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
print("Input Question and Text:")
print(decoded_input)
print("-" * 20) # Separator

# Prepare input_ids for the model
input_ids = torch.tensor([input_ids]).to(device)

# 3. Generate and Decode the Output
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, max_length=256)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 4. Print the Generated and Reference Texts
print("Generated Answer:")
print(generated_text)
reference_text = tokenizer.decode(first_test_example["labels"], skip_special_tokens=True)
print("\nReference Answer:")
print(reference_text)

# 5. (Optional) Calculate Loss
labels = torch.tensor([first_test_example["labels"]]).to(device)
outputs = model(**{"input_ids": input_ids}, labels=labels) # Pass input_ids as a dictionary
loss = outputs.loss
print(f"Calculated Loss: {loss.item()}")

In [12]:
custom_question = "What treaties are being talked about related to the panama canal dispute?"
custom_text = """US President elect Donald Trump has threatened to demand the return of the Panama Canal to the United States if Panama continues to charge what he described as "ridiculous" passage fees for American vessels. In a post on his social media platform Truth social, Trump remarked that "the Panama Canal is considered a VITAL National Asset for the United States, due to its critical role to America's Economy and National Security", and strongly criticised the tolls levied, which can from as low as $0.50 to as high as $300,000. "The fees being charged by Panama are ridiculous, especially knowing the extraordinary generosity that has been bestowed to Panama by the US. This complete 'rip-off' of our country will immediately stop," Trump wrote. The US completed the construction of the canal in 1914 and managed it until December 31, 1999. On that date, control of the canal was officially handed over to Panama, a sovereign country, based on treaties signed in 1997.  The US is the largest user of the Canal. "It was not given for the benefit of others, but merely as a token of cooperation with us and Panama. If the principles, both moral and legal, of this magnanimous gesture of giving are not followed, then we will demand that the Panama Canal be returned to us, in full, and without question," Trump added. """
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. Load Your Fine-tuned Model and Tokenizer
model_path = "./best_model"  # Path to your saved model directory
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Combine question and text into a single input string
input_text = f"Answer the following question based on the provided text:\n\nQuestion: {custom_question} \n\nText: {custom_text}"

# 3. Tokenize the Input
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)

# 4. Generate and Decode the Answer
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=256,
        num_beams=4,
    )
generated_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# 5. Print the Generated Answer
print("Generated Answer:")
print(generated_answer)


Generated Answer:
The treaties are being talked about related to the panama canal dispute. The US is the largest user of the Canal. It was not given for the benefit of others, but merely as a token of cooperation with us and Panama. If the principles, both moral and legal, of this magnanimous gesture of giving are not followed, then we will demand that the Panama Canal be returned to us, in full, and without question.


In [ ]:
for i in range(3):
    print(f"--- Training Example {i+1} ---")
    # print(f"Input IDs: {train_dataset[i]['input_ids']}")
    print(f"Decoded Input: {tokenizer.decode(train_dataset[i]['input_ids'])}")
    # print(f"Label IDs: {train_dataset[i]['labels']}")
    print(f"Decoded Label: {tokenizer.decode(train_dataset[i]['labels'])}")

    print(f"--- Validation Example {i+1} ---")
    # print(f"Input IDs: {val_dataset[i]['input_ids']}")
    print(f"Decoded Input: {tokenizer.decode(val_dataset[i]['input_ids'])}")
    # print(f"Label IDs: {val_dataset[i]['labels']}")
    print(f"Decoded Label: {tokenizer.decode(val_dataset[i]['labels'])}")

    print(f"--- Test Example {i+1} ---")
    # print(f"Input IDs: {test_dataset[i]['input_ids']}")
    print(f"Decoded Input: {tokenizer.decode(test_dataset[i]['input_ids'])}")
    # print(f"Label IDs: {test_dataset[i]['labels']}")
    print(f"Decoded Label: {tokenizer.decode(test_dataset[i]['labels'])}")
    print("\n")